# Entraînement du modèle de classification des prunes africaines

Ce notebook présente le processus d'entraînement du modèle de classification des prunes africaines. Nous utiliserons une approche en deux étapes :
1. **Détection** : Déterminer si l'image contient une prune ou non
2. **Classification** : Si une prune est détectée, classifier son état

## Table des matières

1. [Configuration de l'environnement](#1-configuration-de-lenvironnement)
2. [Chargement des données](#2-chargement-des-données)
3. [Architecture du modèle](#3-architecture-du-modèle)
4. [Entraînement du modèle de détection](#4-entraînement-du-modèle-de-détection)
5. [Entraînement du modèle de classification](#5-entraînement-du-modèle-de-classification)
6. [Évaluation des performances](#6-évaluation-des-performances)
7. [Sauvegarde du modèle](#7-sauvegarde-du-modèle)
8. [Conclusion](#8-conclusion)

## 1. Configuration de l'environnement

Commençons par importer les bibliothèques nécessaires et configurer notre environnement de travail.

In [ ]:
import os
import sys
import time
import json
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

# Ajouter le répertoire parent au chemin pour pouvoir importer nos modules
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

# Importer nos modules personnalisés
from data.data_preprocessing import (
    load_and_prepare_data,
    load_and_prepare_two_stage_data,
    visualize_batch
)
from models.model_architecture import (
    get_model,
    get_two_stage_model,
    PlumClassifier,
    LightweightPlumClassifier,
    TwoStageModel
)

# Définir les chemins des données (à modifier selon votre configuration)
DATA_ROOT = "../data/raw"  # Chemin vers le répertoire de données brutes
PLUM_DATA_DIR = os.path.join(DATA_ROOT, "plums")  # Sous-dossier pour les prunes
NON_PLUM_DATA_DIR = os.path.join(DATA_ROOT, "non_plums")  # Sous-dossier pour les non-prunes
MODELS_DIR = "../models/saved"  # Répertoire pour sauvegarder les modèles entraînés

# Créer les répertoires s'ils n'existent pas
os.makedirs(DATA_ROOT, exist_ok=True)
os.makedirs(PLUM_DATA_DIR, exist_ok=True)
os.makedirs(NON_PLUM_DATA_DIR, exist_ok=True)
os.makedirs(MODELS_DIR, exist_ok=True)

# Définir les paramètres d'entraînement
BATCH_SIZE = 32
IMG_SIZE = 224
NUM_WORKERS = 4
LEARNING_RATE = 0.001
NUM_EPOCHS = 25
EARLY_STOPPING_PATIENCE = 7
RANDOM_SEED = 42

# Fixer les seeds pour la reproductibilité
torch.manual_seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(RANDOM_SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Déterminer le device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Utilisation de: {device}")

## 2. Chargement des données

Chargeons les données préparées pour l'entraînement et la validation de nos modèles.

In [ ]:
# Vérifier si les répertoires de données existent et contiennent des images
def check_data_availability():
    """Vérifie si les données sont disponibles pour l'entraînement."""
    # Vérifier le répertoire des prunes
    plum_classes = [d for d in os.listdir(PLUM_DATA_DIR) if os.path.isdir(os.path.join(PLUM_DATA_DIR, d))]
    if not plum_classes:
        print(f"Aucune classe de prune trouvée dans {PLUM_DATA_DIR}. Veuillez ajouter des données ou créer des exemples.")
        return False
    
    # Vérifier s'il y a des images dans chaque classe
    for cls in plum_classes:
        cls_dir = os.path.join(PLUM_DATA_DIR, cls)
        images = [f for f in os.listdir(cls_dir) 
                 if os.path.isfile(os.path.join(cls_dir, f)) and 
                 f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif'))]
        if not images:
            print(f"Aucune image trouvée dans la classe {cls}. Veuillez ajouter des données.")
            return False
    
    # Vérifier le répertoire des non-prunes
    non_plum_dir = os.path.join(NON_PLUM_DATA_DIR, "non_plum")
    if not os.path.exists(non_plum_dir):
        print(f"Le répertoire {non_plum_dir} n'existe pas. Veuillez créer ce répertoire et y ajouter des images.")
        return False
    
    # Vérifier s'il y a des images dans le répertoire non-prune
    non_plum_images = [f for f in os.listdir(non_plum_dir) 
                      if os.path.isfile(os.path.join(non_plum_dir, f)) and 
                      f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif'))]
    if not non_plum_images:
        print(f"Aucune image trouvée dans {non_plum_dir}. Veuillez ajouter des données.")
        return False
    
    return True

# Vérifier la disponibilité des données
data_available = check_data_availability()

if data_available:
    try:
        # Charger et préparer les données pour les deux étapes
        print("Chargement des données pour les deux étapes...")
        (detection_train_loader, detection_val_loader, detection_test_loader, detection_class_names), \
        (classification_train_loader, classification_val_loader, classification_test_loader, classification_class_names) = \
            load_and_prepare_two_stage_data(
                PLUM_DATA_DIR, 
                NON_PLUM_DATA_DIR,
                batch_size=BATCH_SIZE, 
                img_size=IMG_SIZE,
                num_workers=NUM_WORKERS
            )
        
        print(f"Classes de détection: {detection_class_names}")
        print(f"Classes de classification: {classification_class_names}")
        
        # Afficher les tailles des datasets
        print(f"\nTailles des datasets de détection:")
        print(f"  - Entraînement: {len(detection_train_loader.dataset)} images")
        print(f"  - Validation: {len(detection_val_loader.dataset)} images")
        print(f"  - Test: {len(detection_test_loader.dataset)} images")
        
        print(f"\nTailles des datasets de classification:")
        print(f"  - Entraînement: {len(classification_train_loader.dataset)} images")
        print(f"  - Validation: {len(classification_val_loader.dataset)} images")
        print(f"  - Test: {len(classification_test_loader.dataset)} images")
    except Exception as e:
        print(f"Erreur lors du chargement des données: {e}")
else:
    print("Veuillez d'abord créer des données d'exemple ou ajouter vos propres images.")

## 3. Architecture du modèle

Examinons l'architecture des modèles que nous allons utiliser pour la détection et la classification.

In [ ]:
# Créer et afficher le modèle de détection
def create_detection_model():
    """Crée le modèle de détection (prune vs non-prune)."""
    model = get_model(
        model_name='lightweight', 
        num_classes=2,  # 2 classes: prune et non-prune
        base_model='mobilenet_v2', 
        pretrained=True
    )
    return model

# Créer et afficher le modèle de classification
def create_classification_model(num_classes):
    """Crée le modèle de classification des prunes."""
    model = get_model(
        model_name='standard', 
        num_classes=num_classes,
        base_model='resnet18', 
        pretrained=True
    )
    return model

# Afficher le résumé des modèles
if data_available and 'classification_class_names' in locals():
    # Créer les modèles
    detection_model = create_detection_model()
    classification_model = create_classification_model(len(classification_class_names))
    
    # Afficher les informations sur les modèles
    print("=== Modèle de détection ===")
    print(f"Type: {detection_model.__class__.__name__}")
    print(f"Informations: {detection_model.get_model_info()}")
    
    print("\n=== Modèle de classification ===")
    print(f"Type: {classification_model.__class__.__name__}")
    print(f"Informations: {classification_model.get_model_info()}")
    
    # Afficher le nombre de paramètres
    def count_parameters(model):
        return sum(p.numel() for p in model.parameters() if p.requires_grad)
    
    print(f"\nNombre de paramètres entraînables:")
    print(f"  - Modèle de détection: {count_parameters(detection_model):,}")
    print(f"  - Modèle de classification: {count_parameters(classification_model):,}")

## 4. Entraînement du modèle de détection

Entraînons d'abord le modèle de détection qui déterminera si une image contient une prune ou non.

In [ ]:
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, 
                device, num_epochs=25, early_stopping_patience=7, save_dir=MODELS_DIR, model_name="model"):
    """Entraîne le modèle et sauvegarde le meilleur modèle."""
    # Créer le répertoire de sauvegarde s'il n'existe pas
    os.makedirs(save_dir, exist_ok=True)
    
    # Initialiser les variables
    best_val_loss = float('inf')
    best_val_acc = 0.0
    early_stopping_counter = 0
    
    # Historique pour tracer les courbes
    history = {
        'train_loss': [],
        'val_loss': [],
        'train_acc': [],
        'val_acc': [],
        'lr': []
    }
    
    # Heure de début
    start_time = time.time()
    
    # Boucle d'entraînement
    for epoch in range(num_epochs):
        print(f"Époque {epoch+1}/{num_epochs}")
        print('-' * 10)
        
        # Mode entraînement
        model.train()
        running_loss = 0.0
        running_corrects = 0
        
        # Boucle sur les batches d'entraînement
        for inputs, labels in train_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            # Réinitialiser les gradients
            optimizer.zero_grad()
            
            # Forward pass
            with torch.set_grad_enabled(True):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)
                
                # Backward pass et optimisation
                loss.backward()
                optimizer.step()
            
            # Statistiques
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
        
        # Calculer les métriques d'entraînement
        epoch_train_loss = running_loss / len(train_loader.dataset)
        epoch_train_acc = running_corrects.double() / len(train_loader.dataset)
        
        # Mode évaluation
        model.eval()
        running_loss = 0.0
        running_corrects = 0
        
        # Boucle sur les batches de validation
        for inputs, labels in val_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            # Forward pass
            with torch.no_grad():
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)
            
            # Statistiques
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
        
        # Calculer les métriques de validation
        epoch_val_loss = running_loss / len(val_loader.dataset)
        epoch_val_acc = running_corrects.double() / len(val_loader.dataset)
        
        # Ajuster le learning rate
        current_lr = optimizer.param_groups[0]['lr']
        scheduler.step(epoch_val_loss)
        
        # Afficher les métriques
        print(f"Train Loss: {epoch_train_loss:.4f} Acc: {epoch_train_acc:.4f}")
        print(f"Val Loss: {epoch_val_loss:.4f} Acc: {epoch_val_acc:.4f}")
        print(f"Learning Rate: {current_lr:.6f}")
        
        # Mettre à jour l'historique
        history['train_loss'].append(epoch_train_loss)
        history['val_loss'].append(epoch_val_loss)
        history['train_acc'].append(epoch_train_acc.item())
        history['val_acc'].append(epoch_val_acc.item())
        history['lr'].append(current_lr)
        
        # Sauvegarder le meilleur modèle selon la perte de validation
        if epoch_val_loss < best_val_loss:
            print(f"Amélioration de la perte de validation de {best_val_loss:.4f} à {epoch_val_loss:.4f}. Sauvegarde du modèle...")
            best_val_loss = epoch_val_loss
            torch.save(model.state_dict(), os.path.join(save_dir, f'{model_name}_best_loss.pth'))
            early_stopping_counter = 0
        else:
            early_stopping_counter += 1
        
        # Sauvegarder le meilleur modèle selon l'accuracy de validation
        if epoch_val_acc > best_val_acc:
            print(f"Amélioration de l'accuracy de validation de {best_val_acc:.4f} à {epoch_val_acc:.4f}. Sauvegarde du modèle...")
            best_val_acc = epoch_val_acc
            torch.save(model.state_dict(), os.path.join(save_dir, f'{model_name}_best_acc.pth'))
        
        # Early stopping
        if early_stopping_counter >= early_stopping_patience:
            print(f"Early stopping après {early_stopping_counter} époques sans amélioration")
            break
        
        print()
    
    # Temps total d'entraînement
    time_elapsed = time.time() - start_time
    print(f"Entraînement terminé en {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s")
    print(f"Meilleure perte de validation: {best_val_loss:.4f}")
    print(f"Meilleure accuracy de validation: {best_val_acc:.4f}")
    
    # Sauvegarder le dernier modèle
    torch.save(model.state_dict(), os.path.join(save_dir, f'{model_name}_last.pth'))
    
    # Sauvegarder l'historique
    with open(os.path.join(save_dir, f'{model_name}_history.json'), 'w') as f:
        json.dump(history, f)
    
    return history

def plot_training_history(history, save_dir=MODELS_DIR, model_name="model"):
    """Trace les courbes d'entraînement."""
    # Créer le répertoire de sauvegarde s'il n'existe pas
    os.makedirs(save_dir, exist_ok=True)
    
    # Tracer les courbes de perte
    plt.figure(figsize=(12, 4))
    
    # Graphique des pertes
    plt.subplot(1, 2, 1)
    plt.plot(history['train_loss'], label='Train Loss')
    plt.plot(history['val_loss'], label='Validation Loss')
    plt.xlabel('Époque')
    plt.ylabel('Perte')
    plt.legend()
    plt.title('Évolution des pertes')
    
    # Graphique de l'accuracy
    plt.subplot(1, 2, 2)
    plt.plot(history['train_acc'], label='Train Accuracy')
    plt.plot(history['val_acc'], label='Validation Accuracy')
    plt.xlabel('Époque')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.title('Évolution de l\'accuracy')
    
    plt.tight_layout()
    plt.savefig(os.path.join(save_dir, f'{model_name}_training_curves.png'))
    plt.show()
    
    # Tracer l'évolution du learning rate
    plt.figure(figsize=(10, 4))
    plt.plot(history['lr'])
    plt.xlabel('Époque')
    plt.ylabel('Learning Rate')
    plt.title('Évolution du Learning Rate')
    plt.yscale('log')
    plt.savefig(os.path.join(save_dir, f'{model_name}_learning_rate.png'))
    plt.show()

In [ ]:
# Entraîner le modèle de détection
if data_available and 'detection_train_loader' in locals() and 'detection_val_loader' in locals():
    try:
        print("=== Entraînement du modèle de détection ===\n")
        
        # Créer le modèle de détection
        detection_model = create_detection_model()
        
        # Déplacer le modèle sur le device
        detection_model = detection_model.to(device)
        
        # Définir la fonction de perte et l'optimiseur
        detection_criterion = nn.CrossEntropyLoss()
        detection_optimizer = optim.Adam(detection_model.parameters(), lr=LEARNING_RATE)
        
        # Scheduler pour ajuster le learning rate
        detection_scheduler = ReduceLROnPlateau(detection_optimizer, mode='min', factor=0.1, patience=3, verbose=True)
        
        # Entraîner le modèle de détection
        detection_history = train_model(
            detection_model, 
            detection_train_loader, 
            detection_val_loader, 
            detection_criterion, 
            detection_optimizer, 
            detection_scheduler, 
            device, 
            num_epochs=NUM_EPOCHS, 
            early_stopping_patience=EARLY_STOPPING_PATIENCE,
            save_dir=MODELS_DIR,
            model_name="detection"
        )
        
        # Tracer les courbes d'entraînement
        plot_training_history(detection_history, save_dir=MODELS_DIR, model_name="detection")
    except Exception as e:
        print(f"Erreur lors de l'entraînement du modèle de détection: {e}")

## 5. Entraînement du modèle de classification

Maintenant, entraînons le modèle de classification qui déterminera l'état de la prune.

In [ ]:
# Entraîner le modèle de classification
if data_available and 'classification_train_loader' in locals() and 'classification_val_loader' in locals():
    try:
        print("=== Entraînement du modèle de classification ===\n")
        
        # Créer le modèle de classification
        classification_model = create_classification_model(len(classification_class_names))
        
        # Déplacer le modèle sur le device
        classification_model = classification_model.to(device)
        
        # Définir la fonction de perte et l'optimiseur
        classification_criterion = nn.CrossEntropyLoss()
        classification_optimizer = optim.Adam(classification_model.parameters(), lr=LEARNING_RATE)
        
        # Scheduler pour ajuster le learning rate
        classification_scheduler = ReduceLROnPlateau(classification_optimizer, mode='min', factor=0.1, patience=3, verbose=True)
        
        # Entraîner le modèle de classification
        classification_history = train_model(
            classification_model, 
            classification_train_loader, 
            classification_val_loader, 
            classification_criterion, 
            classification_optimizer, 
            classification_scheduler, 
            device, 
            num_epochs=NUM_EPOCHS, 
            early_stopping_patience=EARLY_STOPPING_PATIENCE,
            save_dir=MODELS_DIR,
            model_name="classification"
        )
        
        # Tracer les courbes d'entraînement
        plot_training_history(classification_history, save_dir=MODELS_DIR, model_name="classification")
    except Exception as e:
        print(f"Erreur lors de l'entraînement du modèle de classification: {e}")

## 6. Évaluation des performances

Évaluons les performances de nos modèles sur les ensembles de test.

In [ ]:
def evaluate_model(model, test_loader, criterion, device, class_names, save_dir=MODELS_DIR, model_name="model"):
    """Évalue le modèle sur l'ensemble de test et génère des visualisations."""
    # Créer le répertoire de sauvegarde s'il n'existe pas
    os.makedirs(save_dir, exist_ok=True)
    
    # Mode évaluation
    model.eval()
    running_loss = 0.0
    running_corrects = 0
    
    # Pour la matrice de confusion
    all_preds = []
    all_labels = []
    
    # Boucle sur les batches de test
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        # Forward pass
        with torch.no_grad():
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)
        
        # Statistiques
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
        
        # Collecter les prédictions et les labels pour la matrice de confusion
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())
    
    # Calculer les métriques
    test_loss = running_loss / len(test_loader.dataset)
    test_acc = running_corrects.double() / len(test_loader.dataset)
    
    print(f"Test Loss: {test_loss:.4f} Acc: {test_acc:.4f}")
    
    # Créer la matrice de confusion
    cm = confusion_matrix(all_labels, all_preds)
    
    # Visualiser la matrice de confusion
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
    plt.xlabel('Prédiction')
    plt.ylabel('Vérité')
    plt.title('Matrice de confusion')
    plt.savefig(os.path.join(save_dir, f'{model_name}_confusion_matrix.png'))
    plt.show()
    
    # Générer le rapport de classification
    report = classification_report(all_labels, all_preds, target_names=class_names, output_dict=True)
    
    # Sauvegarder le rapport
    with open(os.path.join(save_dir, f'{model_name}_classification_report.json'), 'w') as f:
        json.dump(report, f, indent=4)
    
    # Visualiser les métriques par classe
    plt.figure(figsize=(12, 6))
    
    # Extraire les métriques par classe
    classes = list(report.keys())[:-3]  # Exclure 'accuracy', 'macro avg', 'weighted avg'
    precision = [report[cls]['precision'] for cls in classes]
    recall = [report[cls]['recall'] for cls in classes]
    f1 = [report[cls]['f1-score'] for cls in classes]
    
    # Créer le graphique
    x = np.arange(len(classes))
    width = 0.25
    
    plt.bar(x - width, precision, width, label='Precision')
    plt.bar(x, recall, width, label='Recall')
    plt.bar(x + width, f1, width, label='F1-score')
    
    plt.xlabel('Classe')
    plt.ylabel('Score')
    plt.title('Métriques par classe')
    plt.xticks(x, classes, rotation=45)
    plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(save_dir, f'{model_name}_metrics_by_class.png'))
    plt.show()
    
    return {
        'test_loss': test_loss,
        'test_acc': test_acc.item(),
        'confusion_matrix': cm.tolist(),
        'classification_report': report
    }

In [ ]:
# Évaluer le modèle de détection
if data_available and 'detection_test_loader' in locals() and 'detection_class_names' in locals():
    try:
        print("=== Évaluation du modèle de détection ===\n")
        
        # Charger le meilleur modèle (selon l'accuracy)
        detection_model = create_detection_model()
        detection_model.load_state_dict(torch.load(os.path.join(MODELS_DIR, 'detection_best_acc.pth')))
        detection_model = detection_model.to(device)
        
        # Évaluer le modèle
        detection_criterion = nn.CrossEntropyLoss()
        detection_metrics = evaluate_model(
            detection_model, 
            detection_test_loader, 
            detection_criterion, 
            device, 
            detection_class_names,
            save_dir=MODELS_DIR,
            model_name="detection"
        )
    except Exception as e:
        print(f"Erreur lors de l'évaluation du modèle de détection: {e}")

In [ ]:
# Évaluer le modèle de classification
if data_available and 'classification_test_loader' in locals() and 'classification_class_names' in locals():
    try:
        print("=== Évaluation du modèle de classification ===\n")
        
        # Charger le meilleur modèle (selon l'accuracy)
        classification_model = create_classification_model(len(classification_class_names))
        classification_model.load_state_dict(torch.load(os.path.join(MODELS_DIR, 'classification_best_acc.pth')))
        classification_model = classification_model.to(device)
        
        # Évaluer le modèle
        classification_criterion = nn.CrossEntropyLoss()
        classification_metrics = evaluate_model(
            classification_model, 
            classification_test_loader, 
            classification_criterion, 
            device, 
            classification_class_names,
            save_dir=MODELS_DIR,
            model_name="classification"
        )
    except Exception as e:
        print(f"Erreur lors de l'évaluation du modèle de classification: {e}")

## 7. Sauvegarde du modèle

Créons et sauvegardons le modèle à deux étapes complet.

In [ ]:
# Créer et sauvegarder le modèle à deux étapes
if data_available and 'detection_class_names' in locals() and 'classification_class_names' in locals():
    try:
        print("=== Création du modèle à deux étapes ===\n")
        
        # Charger les meilleurs modèles
        detection_model = create_detection_model()
        detection_model.load_state_dict(torch.load(os.path.join(MODELS_DIR, 'detection_best_acc.pth')))
        
        classification_model = create_classification_model(len(classification_class_names))
        classification_model.load_state_dict(torch.load(os.path.join(MODELS_DIR, 'classification_best_acc.pth')))
        
        # Créer le modèle à deux étapes
        two_stage_model = TwoStageModel(detection_model, classification_model, detection_threshold=0.7)
        
        # Sauvegarder les informations du modèle
        model_info = {
            'detection_classes': detection_class_names,
            'classification_classes': classification_class_names,
            'model_info': two_stage_model.get_model_info(),
            'img_size': IMG_SIZE,
            'date_created': time.strftime("%Y-%m-%d %H:%M:%S")
        }
        
        with open(os.path.join(MODELS_DIR, 'two_stage_model_info.json'), 'w') as f:
            json.dump(model_info, f, indent=4)
        
        print("Modèle à deux étapes créé et informations sauvegardées.")
        print(f"Classes de détection: {detection_class_names}")
        print(f"Classes de classification: {classification_class_names}")
        print(f"Informations du modèle: {two_stage_model.get_model_info()}")
    except Exception as e:
        print(f"Erreur lors de la création du modèle à deux étapes: {e}")

## 8. Conclusion

Dans ce notebook, nous avons entraîné et évalué un modèle à deux étapes pour la classification des prunes africaines. Nous avons :

1. Configuré notre environnement de travail
2. Chargé les données préparées
3. Créé et entraîné un modèle de détection (prune vs non-prune)
4. Créé et entraîné un modèle de classification des prunes
5. Évalué les performances des deux modèles
6. Créé et sauvegardé un modèle à deux étapes complet

Ces étapes nous ont permis de développer un système capable de détecter la présence de prunes dans une image et de classifier leur état. Dans le prochain notebook, nous explorerons comment utiliser ce modèle pour faire des prédictions sur de nouvelles images.

### Prochaines étapes

- Test du modèle sur de nouvelles images
- Optimisation des hyperparamètres
- Déploiement du modèle
- Création d'une interface utilisateur